In [6]:
import pynq
import numpy as np
from time import sleep

u2 = pynq.Device.devices[1]
ol = pynq.Overlay("krnl_dummy.xclbin", device=u2)
krnl = ol.krnl_dummy_1
krnl_map = krnl.register_map

In [3]:
ol?

In [4]:
print(krnl_map)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, AUTO_RESTART=0, AP_CONTINUE=0),
  en = Register(value=0),
  dummy_a = Register(value=0),
  dummy_b = Register(value=0)
}


In [5]:
print(krnl.signature)

(en: 'unsigned int', dummy_a: 'void*', dummy_b: 'void*')


In [7]:
def loop_till_fail():
    count = 0
    time = 0.01
    flag = True
    while flag:
        out = pynq.allocate(shape=(2048,2048), dtype=np.uint32, target=ol.bank0)
        gold = np.random.randint(low=0, high=100, size=(2048,2048), dtype=np.uint32)
        out[:] = gold
        out.sync_to_device()
        sleep(time)
        out.sync_from_device()

        if (np.array_equal(out, gold)):
            del(out)
            del(gold)
            time += 0.0001
            count += 1
        else:
            del(out)
            del(gold)
            flag = False
            #print(f"Corrupted on loop {count} in {time} seconds.")
    return time, count

In [8]:
results = []
loops = []
for _ in range(20):
    time, count = loop_till_fail()
    results.append(time)
    loops.append(count)

avg_time = sum(results) / len(results)
avg_loops = sum(loops) / len(loops)
print(f"Avg of {count} loops in {avg_time}")

Avg of 114 loops in 0.02087499999999994
